## 4.1. 模型构造
让我们回顾一下在“多层感知机的简洁实现”一节中含单隐藏层的多层感知机的实现方法。我们首先构造`Sequential`实例，然后依次添加两个全连接层。其中第一层的输出大小为256，即隐藏层单元个数是256；第二层的输出大小为10，即输出层单元个数是10。我们在上一章的其他 节中也使用了`Sequential`类构造模型。这里我们介绍另外一种基于`tf.keras.Model`类的模型构造方法：它让模型构造更加灵活。

### 4.1.1. 继承tf.keras.Model类来构造模型
`tf.keras.Model`类是`tf.keras`模块里提供的一个模型构造类，我们可以继承它来定义我们想要的模型。下面继承`tf.keras.Model`类构造本节开头提到的多层感知机。这里定义的MLP类重载了`tf.keras.Model`类的`__init__`函数和`call`函数(Keras要求继承Model必须重写call方法)。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
class MLP(keras.Model):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.flatten = keras.layers.Flatten()
        self.hidden = keras.layers.Dense(units=256,activation="relu")
        self.outputs = keras.layers.Dense(units=10)
    
    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def call(self,inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        outputs = self.outputs(x)
        return outputs

以上的`MLP`类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的`backward`函数。

我们可以实例化`MLP`类得到模型变量`net`。下面的代码初始化`net`并传入输入数据`X`做一次前向计算。其中，`net(X)`将调用MLP类定义的`call`函数来完成前向计算。

In [3]:
X = tf.random.uniform(shape=(2,20))
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-2.9255956e-01,  1.4999267e-01, -1.0255910e-03, -1.2824988e-01,
        -1.4918950e-01, -6.1416328e-02, -8.8619903e-02,  2.3840412e-02,
        -4.5775622e-02, -5.2173086e-02],
       [-3.1969893e-01,  2.9083306e-01,  5.9871092e-02, -1.9115220e-01,
        -3.2532978e-01, -1.4488333e-01,  9.2274435e-02,  1.9230035e-01,
        -1.8377590e-01,  1.8007308e-04]], dtype=float32)>

### 4.1.2. Sequential
我们刚刚提到，`tf.keras.Model`类是一个通用的部件。事实上，`Sequential`类继承自`tf.keras.Model`类。当模型的前向计算为简单串联各个层的计算时，可以通过更加简单的方式定义模型。这正是`Sequential`类的目的：它提供`add`函数来逐一添加串联的`layers`子类实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

我们用`Sequential`类来实现前面描述的`MLP`类，并使用随机初始化的模型做一次前向计算。

In [4]:
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=256,activation="relu"),
    keras.layers.Dense(units=10)
])

model(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.09690143, -0.11529872,  0.1656316 ,  0.1956892 ,  0.0456181 ,
         0.02321697,  0.16577692, -0.10113695,  0.22019345,  0.07574692],
       [-0.08217232, -0.21100597,  0.40102625,  0.28367168,  0.15970314,
        -0.13069792,  0.30816153, -0.13216215,  0.46777248, -0.06167901]],
      dtype=float32)>

In [5]:
# 以上代码等价为：
model = keras.Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=256,activation="relu"))
model.add(keras.layers.Dense(units=10))

model(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.15399662, -0.14114428,  0.19083136,  0.30880812,  0.10535289,
        -0.03279374, -0.07127143, -0.05082184,  0.08300696,  0.01158642],
       [ 0.08377486, -0.4197342 ,  0.3099264 ,  0.3944304 ,  0.25152662,
         0.06124836,  0.09636673, -0.17347752, -0.06740998, -0.10952017]],
      dtype=float32)>

### 4.1.3. 构造复杂的模型
虽然`Sequential`类可以使模型构造更加简单，且不需要定义`call`函数，但直接继承`Model`类可以极大地拓展模型构造的灵活性。下面我们构造一个稍微复杂点的网络`FancyMLP`。在这个网络中，我们通过`constant`函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用`tensor`的函数和Python的控制流，并多次调用相同的层。

In [6]:
class FancyMLP(keras.Model):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        # 使用constant创建的随机权重参数不会在训练中被迭代（即常数参数）
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.flatten = keras.layers.Flatten()
        self.dense = keras.layers.Dense(units=20,activation=tf.nn.relu)
    
    def call(self,x):
        x = self.flatten(x)
        # 使用创建的常数参数，以及tf的relu函数和matmul函数
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        # 复用全连接层。等价于两个全连接层共享参数
        x = self.dense(x)
        # 控制流，这里我们需要调用tf.norm函数来返回标量进行比较
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < .8:
            x *= 10
        return tf.reduce_sum(x)

在这个`FancyMLP`模型中，我们使用了常数权重`rand_weight`（注意它不是模型参数）、做了矩阵乘法操作（`tf.matmul`）并重复使用了相同的`Dense`层。下面我们来测试该模型的随机初始化和前向计算。

In [7]:
net = FancyMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=2.9525135>

因为`FancyMLP`和`Sequential`类都是`tf.keras.Model`类的子类，所以我们可以嵌套调用它们。

In [8]:
class NestMLP(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.net = keras.Sequential()
        self.net.add(keras.layers.Flatten())
        self.net.add(keras.layers.Dense(64, activation='relu'))
        self.net.add(keras.layers.Dense(32, activation='relu'))
        self.dense = keras.layers.Dense(16, activation='relu')

    def call(self, x):
        return self.dense(self.net(x))

net = keras.Sequential()
net.add(NestMLP())
net.add(keras.layers.Dense(20))
net.add(FancyMLP())

net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=18.252165>